In [4]:
import pandas as pd

pht = '../../PNC2/pht003445.txt'

df = pd.read_csv(pht, sep='\t+', comment='#')

df

/tmp/ipykernel_107451/91362331.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(pht, sep='\t+', comment='#')


,dbGaP_Subject_ID,SUBJID,INT_NUM,INT_TYPE,Race,Sex,Med_birth_year,Med_Rating,ADD011,ADD012,...,SUB_OTC,SUB_PSY,SUB_STER,SUB_STIM,SUB_TOB,SUB_TRAN,SUB_TOT,Education,Mother_Education,Father_Education
0,791086,600001103037,1,MP,AA,M,1999.0,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,5.0,12.0,NaN
1,791086,600001103037,2,MI,AA,M,1999.0,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,791087,600001676724,1,AP,AA,F,1990.0,2,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,12.0,NaN
3,791088,600003245643,1,YPI,AA,M,2003.0,2,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,15.0,18.0
4,791090,600004963801,1,MP,EA,F,1998.0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,5.0,20.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14636,800198,609993878116,2,MI,EA,M,1998.0,4,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14637,800199,609998081561,1,MP,"EA,HI + OT",M,1998.0,0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,16.0,NaN
14638,800199,609998081561,2,MI,"EA,HI + OT",M,1998.0,0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14639,800200,609999144528,1,MP,EA,F,1999.0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,5.0,20.0,18.0


In [5]:
import pickle
from natsort import natsorted

basedir = '../../ImageNomer/data/anton/cohorts/test'
demoname = f'{basedir}/demographics.pkl'

with open(demoname, 'rb') as f:
    demo = pickle.load(f)
    
subs = natsorted(list(demo['race'].keys()))
print(len(subs))

830


In [36]:
isubs = [int(sub) for sub in subs]
med = df[df['SUBJID'].isin(isubs) & df['Mother_Education'].notna() & (df['INT_NUM'] == 1)]
fed = df[df['SUBJID'].isin(isubs) & df['Father_Education'].notna() & (df['INT_NUM'] == 1)]
(fed['Father_Education'] >= 20).value_counts()

False    721
True      42
Name: Father_Education, dtype: int64

In [47]:
demo['Mother_Education'] = dict()
demo['Father_Education'] = dict()

for sub in subs:
    isub = int(sub)
    me = med[med['SUBJID'] == isub]['Mother_Education']
    if me.count() == 1:
        demo['Mother_Education'][sub] = int(me)
#     else:
#         demo['Mother_Education'][sub] = 8
    fe = fed[fed['SUBJID'] == isub]['Father_Education']
    if fe.count() == 1:
        demo['Father_Education'][sub] = int(fe)
#     else:
#         demo['Father_Education'][sub] = 8
        
print(demo['Mother_Education'])
print(demo['Father_Education'])

{'600009963128': 14, '600018902293': 16, '600031697545': 12, '600038720566': 10, '600039015048': 16, '600039665619': 16, '600045443700': 13, '600052596653': 14, '600053476714': 18, '600054124128': 14, '600059829137': 14, '600061763148': 12, '600062084650': 8, '600085654611': 14, '600098941368': 13, '600109657100': 12, '600110501017': 12, '600110937794': 12, '600116672720': 14, '600138617917': 16, '600173623767': 17, '600185621034': 13, '600209995267': 12, '600210241146': 11, '600210683444': 12, '600259084705': 16, '600262185931': 12, '600263649795': 12, '600282088524': 16, '600294591042': 12, '600294620965': 12, '600307190856': 17, '600325379088': 14, '600333703574': 16, '600367706279': 13, '600377610905': 12, '600405811873': 18, '600409285352': 18, '600431843434': 9, '600460215379': 13, '600484292483': 16, '600515650925': 16, '600547831711': 12, '600557347302': 11, '600561317124': 12, '600587013053': 14, '600607081380': 14, '600635899969': 16, '600641554524': 16, '600643741457': 14, '

In [48]:
pickle.dump(demo, open(demoname, 'wb'))

print('Done')

Done
